In [28]:
import serial, struct, numpy as np
from PIL import Image  
from serial.tools import list_ports


In [6]:
#Alphabetically lists the ports by name
def comslist():
    ports = []
    for i in list_ports.comports():
        try:
            ser = serial.Serial(i.name)
            ser.close()
        except serial.SerialException as e:
            print(e)
        else:
            ports.append(i.name)
    ports.sort()
    return ports

#Finds the desired port using the name eg COM1
def selectcom(port):
    try :
        ser = serial.Serial(port)
    except serial.SerialException as e:
        print(e)
    else:
        return ser
    
print(comslist())


[Errno 2] could not open port cu.BLTH: [Errno 2] No such file or directory: 'cu.BLTH'
[Errno 2] could not open port cu.Bluetooth-Incoming-Port: [Errno 2] No such file or directory: 'cu.Bluetooth-Incoming-Port'
[Errno 2] could not open port cu.usbmodem143101: [Errno 2] No such file or directory: 'cu.usbmodem143101'
[]


In [31]:
def prepare_image(path: str) -> np.ndarray:
    img = Image.open(path).convert("L")

    # --- keep aspect ratio ---
    img.thumbnail((28, 28), Image.Resampling.LANCZOS)      # longest side = 28
    canvas = Image.new("L", (28, 28), 0)                   # black background
    x = (28 - img.width)  // 2
    y = (28 - img.height) // 2
    canvas.paste(img, (x, y))

    return np.asarray(canvas, dtype=np.uint8).flatten()

img = prepare_image("image1.jpg")
arduino = serial.Serial(port='/dev/tty.usbmodem143101', baudrate=115200, timeout=.1) 
arduino.write(b'I')                                  # sync
arduino.write(struct.pack('<I', len(img)))
arduino.write(img.tobytes())

arduino.read_until(b'\n')                              # wait for response

b'image classification model determined category as 6\r\n'

In [32]:
arduino.close()